##### Importando Librerias

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.externals import joblib
from sklearn import preprocessing

##### Cambiando directorio de trabajo

In [2]:
os.chdir(r"C:\Users\Fernando\Documents\STUFF\GALILEO\2-Statistical Learning ML\Final")
os.getcwd()


'C:\\Users\\Fernando\\Documents\\STUFF\\GALILEO\\2-Statistical Learning ML\\Final'

##### Carga de archivos

In [5]:
#Cargando los csvs a dataframe de Pandas
dataset = pd.read_csv('data_titanic_proyecto.csv')

#Mostrando columnas dentro del dataset
dataset.columns

Index(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'passenger_class', 'passenger_sex',
       'passenger_survived'],
      dtype='object')

##### Escogiendo las features a utilizar en el modelo

In [8]:
#Se remueven: 'Ticket', 'Name', 'PassengerId' que no es informacion que ayude a predecir.
#'Cabin' puede tener influencia pero puede obtenerse esta informacion de otras variables como
#'fare' or 'class'
# Se removeran 'Embarked' por simplicidad.

# De las variables a usar, hay 177 registros sin la edad de la persona. Se removeran.
dataset[dataset.Age.isnull() == True]

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
5,6,"Moran, Mr. James",NaN,0,0,330877,8.4583,NaN,Q,Lower,M,N
17,18,"Williams, Mr. Charles Eugene",NaN,0,0,244373,13.0000,NaN,S,Middle,M,Y
19,20,"Masselmani, Mrs. Fatima",NaN,0,0,2649,7.2250,NaN,C,Lower,F,Y
26,27,"Emir, Mr. Farred Chehab",NaN,0,0,2631,7.2250,NaN,C,Lower,M,N
28,29,"O'Dwyer, Miss. Ellen ""Nellie""",NaN,0,0,330959,7.8792,NaN,Q,Lower,F,Y
29,30,"Todoroff, Mr. Lalio",NaN,0,0,349216,7.8958,NaN,S,Lower,M,N
31,32,"Spencer, Mrs. William Augustus (Marie Eugenie)",NaN,1,0,PC 17569,146.5208,B78,C,Upper,F,Y
32,33,"Glynn, Miss. Mary Agatha",NaN,0,0,335677,7.7500,NaN,Q,Lower,F,Y
36,37,"Mamee, Mr. Hanna",NaN,0,0,2677,7.2292,NaN,C,Lower,M,Y
42,43,"Kraeff, Mr. Theodor",NaN,0,0,349253,7.8958,NaN,C,Lower,M,N


##### Validando datos nulos,  cambiando variables categoricas a One Hot encoded

In [24]:
RemoveAgeNAs  = dataset.dropna(subset=['Age'])
X =  RemoveAgeNAs[['passenger_sex', 'Age', 'Fare','Parch',
                     'SibSp','passenger_class']]                
Y = RemoveAgeNAs[['passenger_survived']]

#OneHotEncoding the Passenger_Sex, Class and Survived columns
le = preprocessing.LabelEncoder()
eSex = pd.DataFrame(le.fit_transform(X.iloc[:,0]), columns = ['passenger_sex'])
eClass = pd.DataFrame(le.fit_transform(X.iloc[:,5]), columns = ['passenger_class'])
eSurvived = pd.DataFrame(le.fit_transform(Y.iloc[:,0]), columns = ['passenger_survived'])

#Alligning indexes to X and Y original data
eSex = eSex.set_index(X.index)
eClass = eClass.set_index(X.index)
eSurvived = eSurvived.set_index(Y.index)

#Replacing original Passenger_Sex and Passenger_class cols for the one encoded
X.drop(labels=['passenger_sex', 'passenger_class'], axis="columns", inplace=True)
X['passenger_sex'] = eSex['passenger_sex']
X['passenger_class'] = eClass['passenger_class']

#Replacing original Survived col for the one encoded
Y.drop(labels=['passenger_survived'], axis="columns", inplace=True)
Y['passenger_survived'] = eSurvived['passenger_survived']

C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

##### Separando datos en Entrenamiento (xTr | yTr), Validacion (xVa | yVa) y Prueba (xTs | yTs) 

In [27]:
#Creating Train and Test set.
xTr, xTs, yTr, yTs = train_test_split(X, Y, test_size = 0.2, random_state = 0)
# xTr 569 rows -- xTs # 143 rows
# yTr 569 rows -- yTs # 143 rows

#Creating Validation set from Train set.
xTr, xVa, yTr, yVa = train_test_split(xTr, yTr, test_size = 0.2, random_state = 0)
# xTr & yTr # 455 rows
# xVa & yVa # 114 rows
# xTs & yTs # 143 rows
#features(X) = 6, labels(Y) = 1 col
xTr

,Age,Fare,Parch,SibSp,passenger_sex,passenger_class
585,18.0,79.6500,2,0,0,2
380,42.0,227.5250,0,0,0,2
487,58.0,29.7000,0,0,1,2
53,29.0,26.0000,0,1,0,1
716,38.0,227.5250,0,0,0,2
144,18.0,11.5000,0,0,1,1
424,18.0,20.2125,1,1,1,0
138,16.0,9.2167,0,0,1,0
791,16.0,26.0000,0,0,1,1
249,54.0,26.0000,0,1,1,1


##### BootStrapping

En caso de haber sido necesario,  se hubieran generado N muestras usado bootstrapping con un código similar al siguiente. En este, generamos 4 muestras distintas de tamaño igual a los datos de entrenamiento xTr, usando muestreo aleatorio con reemplazo y almacenando los 4 sets nuevos en un diccionario *boots*.

In [28]:
boots = {}
#BootStrapping - Generating i samples (in this case 4, can do as many as needed from same dataset xTr)
for i in range(1,5):
    new = xTr.sample(frac=1,replace=True) 
    boots[i] = new

##### Creando funciones de almacenamiento

Se crea una variable con el nombre y ubicación del archivo de excel para despliegue del dataframe. Luego creamos un Panda dataframe para almacenar datos de los experimentos.

In [29]:
#Configuring the saving path and name of the Excel sheet with the experiments logs.
savetopath = "C:\\Users\\Fernando\\Documents\\STUFF\\GALILEO\\2-Statistical Learning ML\\Final\\EvaluacionDeExperimentos.xlsx"

#Defining function that will save the -data- received into an excel with name 
def utoExcel(data):
    data.to_excel(savetopath,
                  sheet_name='Sheet1', startrow=0, startcol=0)

#Creating an empty dataframe with to store all logs of the different experiments. We will add any new experiment
#and its data (model, variables used and error metrics) and then use the function utoExcel to save it into an excel.
notebook = pd.DataFrame(columns = ["ConfigurationString","Accuracy","Error","precision","Recall","f1-Score"])
notebook

,ConfigurationString,Accuracy,Error,precision,Recall,f1-Score


##### Creando Función para guardar los datos de cada experimento
Se recibe el modelo, la cadena de configuración, los valores Y reales y los Y predicción del modelo. Se evaluan las métricas de error y se guardan en el "notebook" creado anteriormente. 

Además, se guardan los datos del modelo en un archivo .pkl (a excepción de los modelos creados en Tensorflow, que debe indicarse al llamar la función usando el parámetro tf=FALSE/TRUE.

In [32]:
def logXperiment(fittedM, cStr, vlLbls, y_pred, tf=False):
     #Evaluating Confusion Matrix for Model
    from sklearn.metrics import confusion_matrix  
    m = confusion_matrix(vlLbls, y_pred)
    global notebook
    #Calculating Model Metrics using the False and True Positives/Negatives from confusion_matrix
    allsum = m[0][0] + m[1][1] + m[1][0] + m[0][1]

    acc = (m[0][0]+m[1][1])/allsum
    err = 1- (m[0][0]+m[1][1])/allsum
    pre = m[0][0]/(m[0][0]+m[0][1])
    rec = m[0][0]/(m[0][0]+m[1][0])
    f1 = 2*(rec*pre)/(rec+pre)
    
    #Saving experiments and its performance in the notebook
    notebook = notebook.append(pd.DataFrame([[cStr,acc,err,pre,rec,f1]], columns=notebook.columns), ignore_index=True)
    #Saving pkl file into Logs folder. Skip if it is a model from TensorFlow.
    if tf == False:
        os.chdir(r"C:\Users\Fernando\Documents\STUFF\GALILEO\2-Statistical Learning ML\Final\Logs")
        joblib.dump(fittedM, f'{cStr}.pkl')
    return

##### Creando los modelos
Se crean los modelos y en cada uno, se almacenan cadenas de configuracion de cada experimento incluyendo el modelo, hyper-parámetros y variables usadas. Cada cadena tiene el formato siguiente:

*modelo_lr=#.#_HyperparametrosDelModelo_Var1_Var2_..._VarN*

##### Ejemplo: 
Usando un Modelo de Regresion Logistica con Learning Rate = 0.012, Regularizacion L1, Lambda = 0.1, Variables: Age, Fare, Parch, SibSp, passenger_sex & passenger_class, genera una cadena de configuración como la siguiente:

###### tTF.LR_lr=0.012_L1_Lmd=0.1_Age_Fare_Parch_SibSp_passenger_sex_passenger_class

##### Support Vector Machine

In [54]:
def tSVM(trFeats,trLbls, vlFeats, vlLbls,predPoint=None,c=1, gamma=1):
    #kernel = 'linear'
    from sklearn.svm import SVC
    svclassifier = SVC(kernel='linear', C=c, gamma=gamma)  
    svclassifier.fit(trFeats, trLbls)
    y_pred = svclassifier.predict(vlFeats)
    
    #config_string - If list contains numbers, use myString = ','.join(map(str, myList)) 
    cstring = f'tSVM_c={c}_g={gamma}_{"_".join(xTr.columns.values.tolist())}'
    
   #Logging
    logXperiment(fittedM=svclassifier,cStr=cstring,vlLbls=vlLbls, y_pred=y_pred)
    if predPoint == None:
        return
    else:
        return svclassifier.predict(predPoint)

##### Naive Bayes

In [91]:
def tNB(trFeats,trLbls, vlFeats, vlLbls,predPoint=None):
    #Naive Bayes from scikit
    from sklearn.naive_bayes import GaussianNB
    nb = GaussianNB()
    # Train the model using the xTr y yTr
    nb.fit(trFeats,trLbls)
    
    y_pred = nb.predict(vlFeats)
    
    #config_string - If list contains numbers, use myString = ','.join(map(str, myList)) 
    cstring = f'tNB__{"_".join(xTr.columns.values.tolist())}'
    
   #Logging
    logXperiment(fittedM=nb,cStr=cstring,vlLbls=vlLbls, y_pred=y_pred)
    
    return nb.predict(predPoint)

##### Decision Trees

In [55]:
def tDT(trFeats,trLbls, vlFeats, vlLbls,predPoint=None, plotTree=False):
    #Decision Tree
    from sklearn.tree import DecisionTreeClassifier  
    dtclassifier = DecisionTreeClassifier()  
    dtclassifier.fit(trFeats, trLbls)  
    y_pred = dtclassifier.predict(vlFeats)
    
    #config_string - If list contains numbers, use myString = ','.join(map(str, myList)) 
    cstring = f'tDT__{"_".join(xTr.columns.values.tolist())}'
    
   #Logging
    logXperiment(fittedM=dtclassifier,cStr=cstring,vlLbls=vlLbls, y_pred=y_pred)
    
    #Plotting if needed
    #if plotTree == True:
    #    from sklearn import tree
    #    tree.plot_tree(dtclassifier.fit(trFeats, trLbls))
    #returns last entry in the notebook dataframe
    return dtclassifier.predict(predPoint)

##### Regresión Logistica con TensorFlow

In [56]:
def tLogReg(xTr, xVal,yTr, yVal, nfeats, predPoint=None, epochs=100,lr=0.001, L=2, Lambda=0.01):
    import tensorflow as tf
    tf.reset_default_graph() #Resets the graph
    graph = tf.Graph()#Assigns/Creates graph object into variable = graph
    
    #Defines graph and sets it as default in TF. Graph for a Linear Regression
    with graph.as_default():
        #Defining TF place holders for X (features) and Y(topredict)
        X = tf.placeholder("float",[None, nfeats], name="X") 
        Y = tf.placeholder("float", [None, 1],name="Y")
        
        #Defining TF place holder for the learning rate    
        lrate = tf.Variable(0.001,"float", name="lrate")
        #creates variables. W is a tensor of shape nfeats rows and 1 column (to allow matrix mult with X)
        #X = [z, nfeats]  W = [nfeats, 1]   -- X * W = [z,1]
        W = tf.get_variable("W", [nfeats, 1], initializer = tf.contrib.layers.xavier_initializer())
        B = tf.get_variable("B", [1], initializer = tf.zeros_initializer())
        
        with tf.name_scope("Hypothesis"):
            #Creates and stores the hypothesis function in node
            hyp = tf.nn.sigmoid(tf.add(tf.matmul(X,W, name="Hyp_X_by_W"), B,name="Hyp_XW_plus_B"))
            
        with tf.name_scope("Cost"):
            #Creates and stores the cost function in node Cost
            cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = hyp, labels = Y))
            #applies L1 or L2 depending on L function parameter value
            if L == 2:
                regularizer = tf.nn.l2_loss(W)
            elif L == 1:
                l1regularizer = tf.contrib.layers.l1_regularizer(scale=Lambda, scope=None)
                weights = tf.trainable_variables()
                regularizer = tf.contrib.layers.apply_regularization(l1regularizer, weights)
                
            cost_op = tf.reduce_mean(cost + Lambda * regularizer)

        #with tf.name_scope("Grad_Desc_Opt"):
        #    cost_summary = tf.summary.scalar("CostSummary", cost)
        
        #Specifying the optimizer with learning rate  = lr
        lrate = lr
        optimizer = tf.train.AdamOptimizer(lrate).minimize(cost_op)
        init = tf.global_variables_initializer()
        
    cost_history = np.empty(shape=[1],dtype=float)
    with tf.Session(graph=graph) as sess:
        sess.run(init)
        
        for epoch in range(epochs):
            _, c = sess.run([optimizer, cost_op], feed_dict={X: xTr, Y: yTr})
            if epoch % 20 == 0:
                print(f'Epoch= {epoch} cost_op={c}, W= {sess.run(W)}, b= {sess.run(B)}')
                cost_history = np.append(cost_history, c)
            
        # Calculate the predictions hyp > 0.5?
        y_pred = tf.to_float(tf.greater(hyp, 0.5))
        # Calculate accuracy on the test set
        #accuracy = tf.reduce_mean(tf.to_float(tf.equal(Y, correct_prediction)))
        #print ("Train Accuracy:", accuracy.eval({X: xTr, Y: yTr}))
        #print ("Test Accuracy:", accuracy.eval({X: xTs, Y: yTs}))
        
        #Logging
        cstring = f'tTF.LR_lr={lr}_L{L}_Lmd={Lambda}_{"_".join(xTr.columns.values.tolist())}'
        
        Z = sess.run([W, B],feed_dict={X: xTr, Y: yTr})
        Z = np.asarray(Z, dtype=object)
        Z.dump(f'{cstring}.npy')
                
        logXperiment(fittedM=Z, cStr=cstring, vlLbls=yVal, y_pred=y_pred.eval({X: xVal}), tf=True)
        
        if predPoint == None:
            return
        else:
            return y_pred.eval({X: predPoint})



##### K-Folds
Desde mi punto de vista, esta técnica es una versión completa de la separación de datos originales en set de entrenamiento y prueba. Recordando, nosotros hicimos acá, una separación de 3 sets> Un grupo de datos para entrenar el modelo, otro para validarlo y un último, que aún no usamos, el de prueba.

Sin embargo, este aproach original, puede mejorarse al tomar K diferentes sets de prueba, del mismo tamaño todos, en lugar de solo 1 como lo hicimos acá. Aunque la selección de los datos de prueba la hicimos de forma aleatoria, el crear varios sets diferentes de prueba, nos permite evaluar el modelo con K diferentes sets de prueba y por consiguiente, estar más seguros de la efectividad y funcionalidad del modelo previo a lanzarlo a producción. 

Acá, en lugar de tomar 1 solo grupo para pruebas del 20% del set original de datos, pudimos haber hecho esto 5 veces, y resultado con 5 sets de entrenamiento y prueba y luego hacer las evaluaciones del modelo. El modelo es tan bueno como el promedio de su comportamiento en estos 5 sets.

##### Experimentación y Entrenamiento con Diferentes Hiper-parámetros

Support Vector Machine: Probando valores de gamma y c

In [59]:
#Testing tSVM
for i in np.arange(1, 2, 0.1):
    for j in np.arange(1, 10, 1):
        tSVM(trFeats=xTr,trLbls=yTr, vlFeats=xVa, vlLbls=yVa,c=i, gamma=j)

C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y 

C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y 

C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y 

C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Fernando\Anaconda3\envs\galieo.python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression TF: alternando tipo y valores de regularización

In [52]:
tLogReg(xTr=xTr, xVal=xVa,yTr=yTr, yVal=yVa, nfeats=len(xTr.columns), epochs=200,lr=0.002, 
        L=2, Lambda=0.1)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Epoch= 0 cost_op=0.9334260821342468, W= [[ 0.1808345 ]
 [-0.10749652]
 [ 0.3227378 ]
 [-0.45729735]
 [-0.13891686]
 [-0.1180155 ]], b= [-0.00199993]
Epoch= 20 cost_op=0.9040385484695435, W= [[ 0.13879412]
 [-0.14962612]
 [ 0.28300813]
 [-0.41752103]
 [-0.16459806]
 [-0.08221034]], b= [-0.04405146]
Epoch= 40 cost_op=0.8199207782745361, W= [[ 0.08855045]
 [-0.19985685]
 [ 0.24331173]
 [-0.37957522]
 [-0.21605596]
 [-0.08946923]], b= [-0.09346629]
Epoch= 60 cost_op=0.7425643801689148, W= [[ 0.03880535]
 [-0.24291593]
 [ 0.2077327 ]
 [-0.34210566]
 [-0.26099727]
 [-0.09553716]], b= [-0.1416243]
Epoch= 80 cost_op=0.7182753682136536, W= [[ 0.00582751]
 [-0.26537818]
 [ 0.17662077]
 [-0.305

Arboles de Decisión

In [60]:
#Testing DTs
w = pd.DataFrame([xTs.iloc[25,:]])
tDT(trFeats=xTr,trLbls=yTr, vlFeats=xVa, vlLbls=yVa,plotTree=False, predPoint=w)

array([1])

notebook

In [61]:
notebook

,ConfigurationString,Accuracy,Error,precision,Recall,f1-Score
0,tTF.LR_lr=0.002_L2_Lmd=0.1_Age_Fare_Parch_SibS...,0.573913,0.426087,1.000000,0.573913,0.729282
1,tDT__Age_Fare_Parch_SibSp_passenger_sex_passen...,0.747826,0.252174,0.772727,0.784615,0.778626
2,tSVM_c=1.0_g=1_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
3,tSVM_c=1.0_g=2_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
4,tSVM_c=1.0_g=3_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
5,tSVM_c=1.0_g=4_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
6,tSVM_c=1.0_g=5_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
7,tSVM_c=1.0_g=6_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
8,tSVM_c=1.0_g=7_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732
9,tSVM_c=1.0_g=8_Age_Fare_Parch_SibSp_passenger_...,0.739130,0.260870,0.848485,0.736842,0.788732


##### Carga y Prueba de Modelos Seleccionados

In [63]:
os.chdir(r"C:\Users\Fernando\Documents\STUFF\GALILEO\2-Statistical Learning ML\Final\Logs")
svm = joblib.load('tSVM_c=1_g=0.1_Age_Fare_Parch_SibSp_passenger_sex_passenger_class.pkl')
dt = svm_model = joblib.load('tDT__Age_Fare_Parch_SibSp_passenger_sex_passenger_class.pkl')


array([423, 177, 305, 292, 889, 595, 637,  54, 456, 110, 287, 315, 691,
       811, 861,  15, 239, 644, 394, 795, 131, 346,   9, 477, 606, 469,
       325,  72, 724,  40, 382, 824, 682, 591, 614, 197,   1, 512, 640,
       830, 225, 391, 621, 102, 756, 777, 261, 655, 590, 403, 822, 345,
       666, 218, 772, 729, 446, 442, 570,  35, 332, 631, 312, 441, 504,
       251, 609, 812, 297,  97, 212, 390, 417, 558, 195, 681, 677,  67,
       422, 658, 803, 599, 339, 506,  11,  89, 616, 885, 318, 626, 151,
       296, 649, 787, 829, 850, 412, 702, 810, 657, 142, 430, 706, 583,
       291,  66, 310, 544, 779, 266,  51, 620,  63, 450, 438, 769,  21,
       860,  98,  84, 715, 874,  69, 355, 262, 518, 618, 771, 818, 327,
       366, 338, 377, 701, 607, 421, 215, 448, 308, 379, 123, 471, 615],
      dtype=int64)

In [90]:
predictions = pd.DataFrame({'passenger#':np.asarray(xTs.index),'SVM':svm.predict(xTs),'DT':dt.predict(xTs)})
predictions

,passenger#,SVM,DT
0,423,1,1
1,177,1,1
2,305,0,1
3,292,0,0
4,889,0,0
5,595,0,0
6,637,0,0
7,54,0,0
8,456,0,0
9,110,0,0


#### Conclusiones y Recomendaciones

- El proyecto aunque incompleto, siento que me dejó bastante experiencia y aclaré cosas que no tenía claras. La mejor forma de aprender es sufriendo un poco, o bastante y explorar y buscar soluciones por uno mismo.


- Considero que hice no menos de 500 consultas en Google, desde Tensores, sesiones hasta como combinar columnas en Pandas o hacer hot encoding. Para casi todo encontré respuesta en StackOverflow principalmente.


- Las métricas de error almacenadas en excel, y su respectiva selecció creo que puede aplicarsele un factor o peso a cada métrica sobre 100%. De esta forma, dado que para cada problema los valores de las métricas tienen más o menos importancia, puede darse más peso a Recall por ejemplo que a Error. Se puede entonces agregar una columna en el dataframe con esta ponderación calculada y rankear luego para obtener los mejores N modelos.


- Mi mayor dificultad fue llevar a cabo las predicciones puntuales. Por alguna razón, solo el modelo de Arboles de decisiones aceptó que le enviara un dataframe de Pandas. Los otros 2 modelos, no me aceptaron arrays, matrices o dataframes para enviar la obvervación puntual. El error que obtenía era:


- Tener varios modelos es útil para hacer predicciones más certeras. Es como unir fuerzas para llegar a un consenso. 